In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
#import torch
from pathlib import Path

from src.wsilib import WSIDatabase, WSITileDatabase
#from src.models import get_model_unet, get_model_maskrcnn, get_model_resnet34

# WSI preprocessing

In [4]:
database = WSIDatabase(wsi_dir="data/Lung/109_17_HE")

[INFO]: Found 1 .mrxs files in data/Lung/109_17_HE
Loading annotations...


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:05<00:00,  6.00s/it]


[SUCCESS]: Loaded 1 WSIs with annotations


In [5]:
database.save_coco_dataset(output_dir="data/COCO_datasets/slides", level=8)

Processing images...


100%|██████████| 1/1 [00:01<00:00,  1.23s/it]



[SUCCESS]: Dataset saved to data/COCO_datasets/slides. Number of images: 1


In [19]:
tile_db = WSITileDatabase(
    wsi_dir="data/Lung/109_17_HE",
    tile_size=256,
    level=6
)
tile_db.save_dataset(Path("data/COCO_datasets/tiles"))

[INFO]: Found 1 .mrxs files in data/Lung/109_17_HE


Creating tile containers for each WSI: 100%|██████████| 1/1 [00:00<00:00,  2.41it/s]



[SUCCESS]: Created 1 tile containers


Extracting tiles from 109_17_HE_40x: 1224 tiles [00:01, 642.64 tiles/s]                      
Extracting tiles from containers: 100%|██████████| 1/1 [00:04<00:00,  4.88s/it]



Merging annotations...

[SUCCESS]: Dataset saved to data/COCO_datasets/tiles. Total images: 133
